In [1]:
import tensorflow as tf
import numpy as np
import data
import model
import copy
from IPython.display import clear_output
import os
tf.multiply(2,3)
clear_output()
from model import Encoder, Predictor, Joiner, vocab
from data import DataWarAndPeace
# import warprnnt_tensorflow

In [3]:
file_path = "/home/adminvbdi/Desktop/RNNT-for-Vowel-Complete/data/war_and_peace.txt"
data = DataWarAndPeace(file_path)
train_data, test_data = data.train_data, data.test_data
for i in train_data.take(1):
    print(i[0].shape)
    print(i[1].shape)
    print(i[2])
    print(i[3])

x = copy.deepcopy(i[0])
y = copy.deepcopy(i[1])
input_lengths = copy.deepcopy(i[2])
output_lengths = copy.deepcopy(i[3])


print(x.shape, y.shape, input_lengths.shape, output_lengths.shape)

for count, batch_data in enumerate(train_data):

    x, y = batch_data[0], batch_data[1]
    input_lengths = batch_data[2]
    output_lengths = batch_data[3]


    # test encoder
    num_inputs = len(vocab)+1
    encoder_dim = 64
    encoder = Encoder(num_inputs, encoder_dim)
    testEnc = encoder(x)
    print("Test Encoder: ", testEnc.shape)

    # Test Predictor
    num_outputs = len(vocab)+1
    predictor_dim = 64
    joiner_dim = 64
    predictor = Predictor(num_outputs, predictor_dim, joiner_dim, 0)
    testPred = predictor(y)
    print("Test Predictor: ", testPred.shape)

    # Test Joiner
    joiner = Joiner(71, 74, num_outputs)
    testJoin = joiner([testEnc, testPred])
    print_text = "log_probs shape: {}, labels shape: {}".format(testJoin.shape, y.shape)
    print(print_text)

    testJoin = tf.nn.softmax(testJoin)


    if count == 3:
        print(count)
        break






(4, 71)
(4, 74)
tf.Tensor([48 50 52 49], shape=(4,), dtype=int32)
tf.Tensor([69 71 72 72], shape=(4,), dtype=int32)
(4, 71) (4, 74) (4,) (4,)
Test Encoder:  (4, 71, 64)
Test Predictor:  (4, 75, 64)
log_probs shape: (4, 71, 75, 102), labels shape: (4, 74)
Test Encoder:  (4, 71, 64)
Test Predictor:  (4, 75, 64)
log_probs shape: (4, 71, 75, 102), labels shape: (4, 74)
Test Encoder:  (4, 71, 64)
Test Predictor:  (4, 75, 64)
log_probs shape: (4, 71, 75, 102), labels shape: (4, 74)
Test Encoder:  (4, 71, 64)
Test Predictor:  (4, 75, 64)
log_probs shape: (4, 71, 75, 102), labels shape: (4, 74)
3
